# Coursera Kaggle Final Project - Data Science Sales Prediction Competition

***

This challenge serves as final project for the "How to win a data science competition" Coursera course.
the link to the contest can be found here: https://www.kaggle.com/c/competitive-data-science-predict-future-sales/

In this competition, it will be worked with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest Russian software firms - 1C Company. 

Will be predicted total sales for every product and store in the next month. By solving this competition you will be able to apply and enhance your data science skills.


## About Dataset


<p><strong>sales_train.csv --></strong> The training set. Daily historical data from January 2013 to October 2015.<br>
<strong>test.csv --></strong>The test set. Data to forecast the sales for these shops and products for November 2015. The range of the target is [0-20]</p>

In [ ]:
# Importing Libraries

import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
#from sklearn import preprocessing
%matplotlib inline

Loading dataset

In [ ]:
df = pd.read_csv("sales_train.csv")
test_df = pd.read_csv("test.csv")

First glimpse of the data

In [ ]:
print("__________INFORMATION__________")
print(df.info())
print("__________MISSING VALUES_________")
print(df.isnull().sum())
print("__________NULL VALUES__________")
print(df.isna().sum())
print("__________SHAPE__________")
print(df.shape)
print("__________FIRST ROWS__________")
df.head(10)

In [ ]:
df.describe()

In [ ]:
print('# of dupes is:', len(df[df.duplicated()]))

## Preprocessing

Turning date column format into date_type and splitting into year, month, day 

In [ ]:
df['date'] = pd.to_datetime(df['date'], format = '%d.%m.%Y')
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
print(df.shape)
df.head(5)

There are other complementary tables for the project, such as item categories, item names and shope names (which won't be of any help here). For the purpose of the project we will only include the item category, which can be extracted from the file items.csv

In [ ]:
df_items = pd.read_csv("items.csv")
df_items.head()

In [ ]:
df = pd.merge(df, df_items, how = 'left', on = 'item_id')
print(df.shape)
df.head()

In [ ]:
df = df.drop(columns=['item_name'])
df.info()

### Downcasting DataFrame

this method is done in order to save memory

In [ ]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

df = downcast_dtypes(df)
print(df.info())

## EDA - Viz

First we are going to explore the different scatter plots for the main features 

In [ ]:
plt.scatter(df["item_price"],df["item_cnt_day"], alpha=0.1)
plt.xlabel("Item Quantity")
plt.ylabel("Item Price")
#plt.xscale('log')
plt.show

In [ ]:
green_diamond = dict(markerfacecolor='g', marker='D')
fig1 = plt.figure(1, figsize=(9, 6))
ax1 = fig1.add_subplot(111)
ax1.set_yscale("log")
ax1.boxplot(df.item_price, flierprops=green_diamond)


In [ ]:
fig2 = plt.figure(1, figsize=(9, 6))
ax2 = fig2.add_subplot(111)
ax2.set_yscale("log")
ax2.boxplot(df.item_cnt_day, flierprops=green_diamond)

From the boxplots above, we can see some outliers that can make our model representation to overfit, therefore, for this analysis we are going to remove them:

In [ ]:
df = df[df['item_price']<100000]
df = df[df['item_cnt_day']<1000]
df[['item_price','item_cnt_day']].describe()

In [ ]:
fig3 = plt.figure(figsize=(12, 8))
plt.matshow(df.corr(), fignum=fig3.number)
plt.xticks(range(df.shape[1]), df.columns, fontsize=12, rotation=45)
plt.yticks(range(df.shape[1]), df.columns, fontsize=12)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
#plt.title('Correlation Matrix', fontsize=16);

From the correlation matrix we can observe that features shop_id & year are some how correlated

### Tips from instructors #1:

A good exercise is to reproduce previous_value_benchmark. As the name suggest - in this benchmark for the each shop/item pair our predictions are just monthly sales from the previous month, i.e. October 2015.

The most important step at reproducing this score is correctly aggregating daily data and constructing monthly sales data frame. You need to get lagged values, fill NaNs with zeros and clip the values into [0,20] range. If you do it correctly, you'll get precisely 1.16777 on the public leaderboard.

Generating features like this is a necessary basis for more complex models. Also, if you decide to fit some model, don't forget to clip the target into [0,20] range, it makes a big difference.

## Grouping Data for the predictions

We need to create a dataframe (grid) of all the combinations of unique (disctinct) date_block_num, store_id and item_id.

In [ ]:
from itertools import product
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = []
for block_num in df['date_block_num'].unique():
    cur_shops = df.loc[df['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = df.loc[df['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
grid

... and now we need to merge the grid with sales to obtain the monthly sales DF, replacing NA's with 0's for the month without any sales:


In [ ]:
sales_m = df.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': 'sum','item_price': np.mean}).reset_index()
sales_m = pd.merge(grid,sales_m,on=['date_block_num','shop_id','item_id'],how='left').fillna(0)

# adding the category id too from the items table.
sales_m = pd.merge(sales_m,df_items,on=['item_id'],how='left')
sales_m.head(10)

In [ ]:
sales_m.describe()

### Tips from instructors #2:

You can get a rather good score after creating some lag-based features like in advice from previous week and feeding them into gradient boosted trees model.

Apart from item/shop pair lags you can try adding lagged values of total shop or total item sales (which are essentially mean-encodings). All of that is going to add some new information.

## Feature Engineering

### - Target Encodings

First we are going to apply the learnings from the feature engineering class of the course, thus, we will apply first some target encodings: MEAN for item_price & item_cnt_day and SUM for item_cnt_day

In [ ]:
feats = ['item_id','shop_id','item_category_id']

agglist = [('item_price',np.mean,'avg'),('item_cnt_day',np.sum,'sum'),('item_cnt_day',np.mean,'avg')]

for ids in feats:
    for column_id,operation,aggtype in agglist:
        # get numbers from sales data and set column names
        mean_df = sales_m.groupby([ids,'date_block_num']).aggregate(operation).reset_index()[[column_id,ids,'date_block_num']]
        mean_df.columns = [ids+'_'+aggtype+'_'+column_id,ids,'date_block_num']
        # merge new columns on sales_m data
        sales_m = pd.merge(sales_m,mean_df,on=['date_block_num',ids],how='left')

In [ ]:
sales_m.head()

### - Lag Features

In [ ]:
lag_variables  = ['item_id_avg_item_price','item_id_sum_item_cnt_day','item_id_avg_item_cnt_day','shop_id_avg_item_price','shop_id_sum_item_cnt_day','shop_id_avg_item_cnt_day','item_category_id_avg_item_price','item_category_id_sum_item_cnt_day','item_category_id_avg_item_cnt_day','item_cnt_day']
lags = [1 ,2 ,3 ,10 ,11 ,12]
# we will keep the results in thsi dataframe
sales_means = sales_m.copy()
for lag in lags:
    sales_new_df = sales_m.copy()
    sales_new_df.date_block_num+=lag
    # subset only the lag variables we want
    sales_new_df = sales_new_df[['date_block_num','shop_id','item_id']+lag_variables]
    sales_new_df.columns = ['date_block_num','shop_id','item_id']+ [lag_feat+'_lag_'+str(lag) for lag_feat in lag_variables]
    # join with date_block_num,shop_id and item_id
    sales_means = pd.merge(sales_means, sales_new_df,on=['date_block_num','shop_id','item_id'] ,how='left')

#### - Random Forest

In [ ]:
# RandomForest
from sklearn import ensemble
model = ensemble.RandomForestClassifier(n_estimators=200, n_jobs=-1, verbose=2, random_state=0)
model.fit(train_df, ytrain)
preds = model.predict_proba(valid_df)[:, 1]


#### - LSTM

In [ ]:
# LSTM
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)
regressor.summary()

#Predicting
predicted_stock_price = regressor.predict(X_test)

### Metric: RMSE

In [ ]:
#RMSE
from sklearn.metrics import mean_squared_error
RMSE = mean_squared_error(y_true, y_pred, squared=False)

In [ ]:
#preds.to_csv("Submissions/submission_#.csv")

#REFERENCES
https://towardsdatascience.com/time-series-machine-learning-regression-framework-9ea33929009a
https://towardsdatascience.com/how-not-to-use-machine-learning-for-time-series-forecasting-avoiding-the-pitfalls-19f9d7adf424
    